In [1]:
# import Pkg; Pkg.add("Tables"); Pkg.add("MarkdownTables"); Pkg.add("DataFrames")

In [1]:
include("datasetloader.jl")
include("benchmark.jl")
using LanguageIdentification
import LanguageIdentification as LI
using BSON

In [6]:
WV = WikiDataSet("corpus/wikipedia/test", langs=LI.supported_languages())
# TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LI.supported_languages())

10000-element WikiDataSet:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## vocabsize tatoeba

In [ ]:
path = "benchmarks/v1"
mkpath(path)
bmkmats = []
log_file = "$path/bmkmats-vocabsize-tatoeba.md"
last_vocsize = 0
redirect_stdio(stdout=log_file) do
    for n in 1:7
        for sz in [100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000]
            LI.initialize(ngram=n, cutoff=2, vocabulary_size=sz)
            vocsize = sum(last.(LI.vocabulary_sizes()))
            if vocsize == last_vocsize
                continue
            end
            global last_vocsize = vocsize
            println("# $n-grams $sz-vocab; total vocab: $vocsize")
            r = benchmark((n, sz) => langid, dataset=TV, languages=LI.supported_languages())
            push!(bmkmats, r)
            showtable(["$n-grams $sz vocab" => last(only(r))], LI.supported_languages())
            bson("$path/bmkmats-vocabsize-tatoeba.bson", bmkmats=bmkmats)
            flush(stdout)
        end
    end
end

## vocabsize wikipedia

In [8]:
path = "benchmarks/v1"
mkpath(path)
bmkmats = []
log_file = "$path/bmkmats-vocabsize-wikipedia.md"
last_vocsize = 0
redirect_stdio(stdout=log_file) do
    for n in 1:7
        for sz in [100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000]
            LI.initialize(ngram=n, cutoff=2, vocabulary_size=sz)
            vocsize = sum(last.(LI.vocabulary_sizes()))
            println("# $n-grams $sz-vocab; total vocab: $vocsize")
            if vocsize == last_vocsize
                continue
            end
            global last_vocsize = vocsize
            r = benchmark((n, sz) => langid, dataset=WV, languages=LI.supported_languages())
            push!(bmkmats, r)
            showtable(["$n-grams $sz vocab" => last(only(r))], LI.supported_languages())
            bson("$path/bmkmats-vocabsize-wikipedia.bson", bmkmats=bmkmats)
            flush(stdout)
        end
    end
end

## cutoff wikipedia

In [ ]:
path = "benchmarks/v1"
mkpath(path)
bmkmats = []
log_file = "$path/bmkmats-cutoff-wikipedia.md"
last_vocsize = 0
redirect_stdio(stdout=log_file) do
    for n in 1:7
        for cf in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            LI.initialize(ngram=n, cutoff=cf, vocabulary_size=1000000)
            vocsize = sum(last.(LI.vocabulary_sizes()))
            println("# $n-grams $cf cutoff; total vocab: $vocsize")
            if vocsize == last_vocsize
                continue
            end
            global last_vocsize = vocsize
            r = benchmark((n, sz) => langid, dataset=WV, languages=LI.supported_languages())
            push!(bmkmats, r)
            showtable(["$n-grams $cf cutoff" => last(only(r))], LI.supported_languages())
            bson("$path/bmkmats-cutoff-wikipedia.bson", bmkmats=bmkmats)
            flush(stdout)
        end
    end
end

## tatoeba

In [7]:
TB = benchmark(
"detector_ngram1"=>t->detector_lh(t, ngram=1),
"detector_ngram2"=>t->detector_lh(t, ngram=2),
"detector_ngram3"=>t->detector_lh(t, ngram=3),
"detector_ngram4"=>t->detector_lh(t, ngram=4),
"detector_ngram5"=>detector_lh,
# "Languages.jl"=>detector_ls, 
# "LanguageDetect.jl"=>detector_ld,
 dataset=TV, languages=LANGS)

detector_ngram1

 32.252705 seconds (40.47 M allocations: 3.161 GiB, 1.49% gc time, 0.63% compilation time)
detector_ngram2

 99.084141 seconds (63.23 M allocations: 5.854 GiB, 0.85% gc time, 0.06% compilation time)
detector_ngram3

196.862576 seconds (85.08 M allocations: 8.258 GiB, 0.60% gc time, 0.03% compilation time)


detector_ngram4298.163596 seconds (106.08 M allocations: 10.882 GiB, 0.54% gc time, 0.03% compilation time)
detector_ngram5

340.773205 seconds (126.62 M allocations: 14.058 GiB, 0.46% gc time, 0.02% compilation time)


5-element Vector{Any}:
 "detector_ngram1" => [0.9949579831932773, 0.9632892804698973, 1.0, 0.7443729903536977, 0.40865384615384615, 0.7161716171617162, 0.5386465132256957, 0.6318933675927413, 1.0, 0.7644653534998241  …  0.22999547352009253, 0.8540268456375839, 0.6937925813777441, 0.8863955119214586, 0.7868983957219251, 0.8710210830921868, 0.8196257615317668, 0.9974704890387859, 0.9356321839080459, 1.0]
 "detector_ngram2" => [0.9932773109243698, 0.9706314243759178, 1.0, 0.7966237942122186, 0.7716346153846154, 0.8726622662266227, 0.7159051872208863, 0.9170065842299663, 1.0, 0.9426442138586001  …  0.7294170899763617, 0.9496644295302014, 0.8690386071158214, 0.9046283309957924, 0.946524064171123, 0.9631803775664876, 0.8133159268929504, 0.9983136593591906, 0.9770114942528736, 1.0]
 "detector_ngram3" => [0.9915966386554622, 0.9765051395007343, 1.0, 0.8215434083601286, 0.8966346153846154, 0.91996699669967, 0.8069391961525249, 0.975590171832343, 1.0, 0.9819293000351741  …  0.8916662475481567, 0

In [8]:
showtable(TB, LANGS)

|                 | Average | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb    | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nds    | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-----------------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
| detector_ngram1 |  73.85% | 99

|                 | Average | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb    | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nds    | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-----------------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
| detector_ngram1 |  73.85% | 99.50% | 96.33% | 100.00% | 74.44% | 40.87% | 71.62% | 53.86% | 63.19% | 100.00% | 76.45% | 62.98% | 96.91% | 88.16% | 48.80% | 60.68% |  8.94% | 98.84% | 81.23% | 88.05% | 25.89% |  5.25% | 91.55% | 36.89% | 99.82% | 93.27% | 100.00% | 98.56% | 84.39% | 92.96% | 82.78% | 72.86% | 63.07% | 40.15% | 60.33% | 35.92% | 97.17% | 59.61% | 84.12% | 76.28% | 73.14% | 23.00% | 85.40% | 69.38% | 88.64% | 78.69% | 87.10% | 81.96% | 99.75% | 93.56% | 100.00% |
| detector_ngram2 |  88.79% | 99.33% | 97.06% | 100.00% | 79.66% | 77.16% | 87.27% | 71.59% | 91.70% | 100.00% | 94.26% | 87.41% | 98.02% | 97.85% | 88.60% | 95.33% |  9.27% | 98.57% | 85.30% | 97.06% | 76.17% | 66.34% | 97.44% | 86.93% | 99.95% | 98.61% | 100.00% | 99.36% | 93.30% | 98.09% | 84.88% | 83.00% | 91.90% | 80.86% | 91.86% | 71.38% | 98.90% | 90.05% | 91.02% | 83.50% | 85.49% | 72.94% | 94.97% | 86.90% | 90.46% | 94.65% | 96.32% | 81.33% | 99.83% | 97.70% | 100.00% |
| detector_ngram3 |  92.99% | 99.16% | 97.65% | 100.00% | 82.15% | 89.66% | 92.00% | 80.69% | 97.56% | 100.00% | 98.19% | 96.56% | 98.60% | 98.89% | 96.55% | 98.17% | 10.53% | 98.77% | 85.43% | 98.65% | 88.18% | 88.53% | 98.94% | 96.21% | 99.97% | 99.39% | 100.00% | 99.52% | 95.50% | 99.25% | 84.85% | 89.40% | 96.05% | 93.40% | 96.41% | 80.88% | 99.32% | 96.45% | 95.29% | 89.36% | 89.83% | 89.17% | 97.99% | 93.38% | 91.73% | 97.94% | 98.58% | 86.30% | 99.92% | 98.62% | 100.00% |
| detector_ngram4 |  94.24% | 99.09% | 97.50% | 100.00% | 85.85% | 91.11% | 93.07% | 85.19% | 98.93% | 100.00% | 99.04% | 98.60% | 98.90% | 98.99% | 97.93% | 98.99% | 14.37% | 98.89% | 86.61% | 99.01% | 91.18% | 94.47% | 99.25% | 97.58% | 99.97% | 99.47% | 100.00% | 99.68% | 94.73% | 99.51% | 86.41% | 92.55% | 97.40% | 96.26% | 97.56% | 83.08% | 99.42% | 97.82% | 96.61% | 91.44% | 91.20% | 92.72% | 98.24% | 95.50% | 91.87% | 98.88% | 98.94% | 89.19% | 99.92% | 99.20% | 100.00% |
| detector_ngram5 |  94.58% | 99.03% | 97.36% | 100.00% | 86.50% | 92.31% | 93.32% | 85.61% | 99.09% | 100.00% | 99.10% | 98.69% | 99.12% | 98.99% | 98.12% | 99.08% | 19.80% | 99.02% | 89.50% | 99.02% | 90.99% | 94.89% | 99.10% | 97.60% | 99.97% | 99.47% | 100.00% | 99.52% | 94.52% | 99.59% | 87.15% | 93.29% | 97.88% | 96.37% | 97.64% | 83.08% | 99.49% | 97.87% | 96.67% | 92.23% | 91.20% | 92.91% | 98.15% | 95.87% | 92.01% | 98.98% | 98.95% | 90.71% | 99.92% | 99.20% | 100.00% |


In [9]:
showtable(TB, LANGS, threshold=0.)

50 columns in total, 50 columns left.
|                 | Average | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb    | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nds    | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-----------------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----

|                 | Average | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb    | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nds    | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-----------------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
| detector_ngram1 |  73.85% | 99.50% | 96.33% | 100.00% | 74.44% | 40.87% | 71.62% | 53.86% | 63.19% | 100.00% | 76.45% | 62.98% | 96.91% | 88.16% | 48.80% | 60.68% |  8.94% | 98.84% | 81.23% | 88.05% | 25.89% |  5.25% | 91.55% | 36.89% | 99.82% | 93.27% | 100.00% | 98.56% | 84.39% | 92.96% | 82.78% | 72.86% | 63.07% | 40.15% | 60.33% | 35.92% | 97.17% | 59.61% | 84.12% | 76.28% | 73.14% | 23.00% | 85.40% | 69.38% | 88.64% | 78.69% | 87.10% | 81.96% | 99.75% | 93.56% | 100.00% |
| detector_ngram2 |  88.79% | 99.33% | 97.06% | 100.00% | 79.66% | 77.16% | 87.27% | 71.59% | 91.70% | 100.00% | 94.26% | 87.41% | 98.02% | 97.85% | 88.60% | 95.33% |  9.27% | 98.57% | 85.30% | 97.06% | 76.17% | 66.34% | 97.44% | 86.93% | 99.95% | 98.61% | 100.00% | 99.36% | 93.30% | 98.09% | 84.88% | 83.00% | 91.90% | 80.86% | 91.86% | 71.38% | 98.90% | 90.05% | 91.02% | 83.50% | 85.49% | 72.94% | 94.97% | 86.90% | 90.46% | 94.65% | 96.32% | 81.33% | 99.83% | 97.70% | 100.00% |
| detector_ngram3 |  92.99% | 99.16% | 97.65% | 100.00% | 82.15% | 89.66% | 92.00% | 80.69% | 97.56% | 100.00% | 98.19% | 96.56% | 98.60% | 98.89% | 96.55% | 98.17% | 10.53% | 98.77% | 85.43% | 98.65% | 88.18% | 88.53% | 98.94% | 96.21% | 99.97% | 99.39% | 100.00% | 99.52% | 95.50% | 99.25% | 84.85% | 89.40% | 96.05% | 93.40% | 96.41% | 80.88% | 99.32% | 96.45% | 95.29% | 89.36% | 89.83% | 89.17% | 97.99% | 93.38% | 91.73% | 97.94% | 98.58% | 86.30% | 99.92% | 98.62% | 100.00% |
| detector_ngram4 |  94.24% | 99.09% | 97.50% | 100.00% | 85.85% | 91.11% | 93.07% | 85.19% | 98.93% | 100.00% | 99.04% | 98.60% | 98.90% | 98.99% | 97.93% | 98.99% | 14.37% | 98.89% | 86.61% | 99.01% | 91.18% | 94.47% | 99.25% | 97.58% | 99.97% | 99.47% | 100.00% | 99.68% | 94.73% | 99.51% | 86.41% | 92.55% | 97.40% | 96.26% | 97.56% | 83.08% | 99.42% | 97.82% | 96.61% | 91.44% | 91.20% | 92.72% | 98.24% | 95.50% | 91.87% | 98.88% | 98.94% | 89.19% | 99.92% | 99.20% | 100.00% |
| detector_ngram5 |  94.58% | 99.03% | 97.36% | 100.00% | 86.50% | 92.31% | 93.32% | 85.61% | 99.09% | 100.00% | 99.10% | 98.69% | 99.12% | 98.99% | 98.12% | 99.08% | 19.80% | 99.02% | 89.50% | 99.02% | 90.99% | 94.89% | 99.10% | 97.60% | 99.97% | 99.47% | 100.00% | 99.52% | 94.52% | 99.59% | 87.15% | 93.29% | 97.88% | 96.37% | 97.64% | 83.08% | 99.49% | 97.87% | 96.67% | 92.23% | 91.20% | 92.91% | 98.15% | 95.87% | 92.01% | 98.98% | 98.95% | 90.71% | 99.92% | 99.20% | 100.00% |


In [10]:
showtable(TB, LANGS, threshold=0.5)

50 columns in total, 41 columns left.
|                 | Average | ara    | bel    | ben     | bul    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | hau    | heb    | hin    | hun    | isl    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nld    | pol    | por    | ron    | rus    | slk    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-----------------|---------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
| detector_ngram1 |  83.58% | 99.50% | 96.33% | 100.00% | 74.44% | 71.62% | 53.86% | 63.19% | 100.00% | 76.45% | 62.98% | 96.91% | 88.16% | 60.68% | 98.84% 

|                 | Average | ara    | bel    | ben     | bul    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | hau    | heb    | hin    | hun    | isl    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nld    | pol    | por    | ron    | rus    | slk    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-----------------|---------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
| detector_ngram1 |  83.58% | 99.50% | 96.33% | 100.00% | 74.44% | 71.62% | 53.86% | 63.19% | 100.00% | 76.45% | 62.98% | 96.91% | 88.16% | 60.68% | 98.84% | 81.23% | 88.05% | 91.55% | 99.82% | 93.27% | 100.00% | 98.56% | 84.39% | 92.96% | 82.78% | 72.86% | 63.07% | 60.33% | 97.17% | 59.61% | 84.12% | 76.28% | 73.14% | 85.40% | 69.38% | 88.64% | 78.69% | 87.10% | 81.96% | 99.75% | 93.56% | 100.00% |
| detector_ngram2 |  92.92% | 99.33% | 97.06% | 100.00% | 79.66% | 87.27% | 71.59% | 91.70% | 100.00% | 94.26% | 87.41% | 98.02% | 97.85% | 95.33% | 98.57% | 85.30% | 97.06% | 97.44% | 99.95% | 98.61% | 100.00% | 99.36% | 93.30% | 98.09% | 84.88% | 83.00% | 91.90% | 91.86% | 98.90% | 90.05% | 91.02% | 83.50% | 85.49% | 94.97% | 86.90% | 90.46% | 94.65% | 96.32% | 81.33% | 99.83% | 97.70% | 100.00% |
| detector_ngram3 |  95.52% | 99.16% | 97.65% | 100.00% | 82.15% | 92.00% | 80.69% | 97.56% | 100.00% | 98.19% | 96.56% | 98.60% | 98.89% | 98.17% | 98.77% | 85.43% | 98.65% | 98.94% | 99.97% | 99.39% | 100.00% | 99.52% | 95.50% | 99.25% | 84.85% | 89.40% | 96.05% | 96.41% | 99.32% | 96.45% | 95.29% | 89.36% | 89.83% | 97.99% | 93.38% | 91.73% | 97.94% | 98.58% | 86.30% | 99.92% | 98.62% | 100.00% |
| detector_ngram4 |  96.42% | 99.09% | 97.50% | 100.00% | 85.85% | 93.07% | 85.19% | 98.93% | 100.00% | 99.04% | 98.60% | 98.90% | 98.99% | 98.99% | 98.89% | 86.61% | 99.01% | 99.25% | 99.97% | 99.47% | 100.00% | 99.68% | 94.73% | 99.51% | 86.41% | 92.55% | 97.40% | 97.56% | 99.42% | 97.82% | 96.61% | 91.44% | 91.20% | 98.24% | 95.50% | 91.87% | 98.88% | 98.94% | 89.19% | 99.92% | 99.20% | 100.00% |
| detector_ngram5 |  96.65% | 99.03% | 97.36% | 100.00% | 86.50% | 93.32% | 85.61% | 99.09% | 100.00% | 99.10% | 98.69% | 99.12% | 98.99% | 99.08% | 99.02% | 89.50% | 99.02% | 99.10% | 99.97% | 99.47% | 100.00% | 99.52% | 94.52% | 99.59% | 87.15% | 93.29% | 97.88% | 97.64% | 99.49% | 97.87% | 96.67% | 92.23% | 91.20% | 98.15% | 95.87% | 92.01% | 98.98% | 98.95% | 90.71% | 99.92% | 99.20% | 100.00% |


## wikipedia

In [11]:
WB = benchmark(
"detector_ngram1"=>t->detector_lh(t, ngram=1),
"detector_ngram2"=>t->detector_lh(t, ngram=2),
"detector_ngram3"=>t->detector_lh(t, ngram=3),
"detector_ngram4"=>t->detector_lh(t, ngram=4),
"detector_ngram5"=>detector_lh,
# "Languages.jl"=>detector_ls, 
# "LanguageDetect.jl"=>detector_ld,
 dataset=WV, languages=LANGS)

detector_ngram1  6.127989 seconds (18.98 M allocations: 1.299 GiB, 3.13% gc time, 1.42% compilation time)
detector_ngram2

 15.790266 seconds (37.25 M allocations: 2.512 GiB, 2.11% gc time, 0.44% compilation time)
detector_ngram3

 34.596629 seconds (55.49 M allocations: 3.933 GiB, 1.41% gc time, 0.18% compilation time)
detector_ngram4

 57.278567 seconds (73.71 M allocations: 5.453 GiB, 1.15% gc time, 0.11% compilation time)
detector_ngram5

 87.003336 seconds (91.91 M allocations: 7.130 GiB, 0.95% gc time, 0.07% compilation time)


5-element Vector{Any}:
 "detector_ngram1" => [0.995, 0.99, 1.0, 0.94, 0.985, 0.935, 0.885, 0.945, 1.0, 0.91  …  0.9, 0.965, 0.95, 0.995, 0.93, 0.985, 0.995, 0.975, 0.925, 1.0]
 "detector_ngram2" => [0.995, 0.99, 1.0, 0.96, 1.0, 0.96, 0.96, 0.96, 1.0, 0.99  …  0.985, 0.975, 0.995, 0.995, 0.98, 0.985, 1.0, 0.97, 0.95, 1.0]
 "detector_ngram3" => [1.0, 0.99, 1.0, 0.98, 0.995, 0.985, 0.97, 0.96, 1.0, 1.0  …  1.0, 0.985, 0.985, 0.995, 0.985, 0.985, 1.0, 0.965, 0.975, 0.995]
 "detector_ngram4" => [1.0, 0.99, 1.0, 0.995, 1.0, 0.98, 0.99, 0.965, 1.0, 1.0  …  1.0, 0.985, 0.985, 0.995, 0.985, 0.985, 1.0, 0.965, 0.985, 0.995]
 "detector_ngram5" => [1.0, 0.995, 1.0, 1.0, 1.0, 0.98, 0.99, 0.97, 1.0, 1.0  …  1.0, 0.99, 0.99, 0.99, 0.985, 0.985, 1.0, 0.965, 0.99, 0.995]

In [12]:
showtable(WB, LANGS)

|                 | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-----------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| de

|                 | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-----------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| detector_ngram1 |  89.29% |  99.50% | 99.00% | 100.00% |  94.00% |  98.50% | 93.50% | 88.50% | 94.50% | 100.00% |  91.00% |  92.00% |  99.50% | 97.50% |  93.00% | 86.00% |  5.50% | 100.00% | 87.00% |  99.00% | 66.50% |  1.50% | 96.50% | 82.00% | 91.00% | 98.00% | 100.00% | 99.00% | 94.00% |  99.00% | 91.00% | 93.00% | 96.00% | 85.00% | 76.50% | 39.50% |  99.00% | 92.00% | 89.00% |  99.50% | 96.50% |  90.00% | 96.50% | 95.00% | 99.50% | 93.00% | 98.50% |  99.50% | 97.50% | 92.50% | 100.00% |
| detector_ngram2 |  94.82% |  99.50% | 99.00% | 100.00% |  96.00% | 100.00% | 96.00% | 96.00% | 96.00% | 100.00% |  99.00% |  97.00% |  99.50% | 99.00% |  99.00% | 97.50% |  7.00% | 100.00% | 89.50% | 100.00% | 84.00% | 71.50% | 98.00% | 97.00% | 96.50% | 98.50% | 100.00% | 99.00% | 95.50% | 100.00% | 91.50% | 95.50% | 99.00% | 97.50% | 95.00% | 85.00% | 100.00% | 97.50% | 90.50% |  99.50% | 96.50% |  98.50% | 97.50% | 99.50% | 99.50% | 98.00% | 98.50% | 100.00% | 97.00% | 95.00% | 100.00% |
| detector_ngram3 |  95.95% | 100.00% | 99.00% | 100.00% |  98.00% |  99.50% | 98.50% | 97.00% | 96.00% | 100.00% | 100.00% |  99.50% | 100.00% | 99.00% | 100.00% | 98.50% |  8.50% | 100.00% | 89.50% | 100.00% | 90.00% | 85.00% | 98.50% | 98.50% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 92.50% | 96.50% | 99.00% | 99.00% | 96.50% | 91.50% | 100.00% | 99.50% | 90.50% |  99.50% | 97.00% | 100.00% | 98.50% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 97.50% |  99.50% |
| detector_ngram4 |  96.89% | 100.00% | 99.00% | 100.00% |  99.50% | 100.00% | 98.00% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.00% | 30.50% | 100.00% | 89.50% |  99.50% | 93.50% | 92.50% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 94.50% | 98.00% | 99.00% | 99.00% | 97.00% | 95.00% |  99.50% | 99.50% | 91.00% | 100.00% | 97.00% | 100.00% | 98.50% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% |  99.50% |
| detector_ngram5 |  98.01% | 100.00% | 99.50% | 100.00% | 100.00% | 100.00% | 98.00% | 99.00% | 97.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.00% | 78.50% | 100.00% | 90.00% |  99.50% | 94.00% | 93.50% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 96.50% | 100.00% | 97.50% | 99.00% | 99.00% | 99.00% | 97.50% | 94.50% |  99.50% | 99.50% | 91.00% | 100.00% | 97.00% | 100.00% | 99.00% | 99.00% | 99.00% | 98.50% | 98.50% | 100.00% | 96.50% | 99.00% |  99.50% |


In [13]:
showtable(WB, LANGS, threshold=0.)

50 columns in total, 50 columns left.
|                 | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-----------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|-----

|                 | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-----------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| detector_ngram1 |  89.29% |  99.50% | 99.00% | 100.00% |  94.00% |  98.50% | 93.50% | 88.50% | 94.50% | 100.00% |  91.00% |  92.00% |  99.50% | 97.50% |  93.00% | 86.00% |  5.50% | 100.00% | 87.00% |  99.00% | 66.50% |  1.50% | 96.50% | 82.00% | 91.00% | 98.00% | 100.00% | 99.00% | 94.00% |  99.00% | 91.00% | 93.00% | 96.00% | 85.00% | 76.50% | 39.50% |  99.00% | 92.00% | 89.00% |  99.50% | 96.50% |  90.00% | 96.50% | 95.00% | 99.50% | 93.00% | 98.50% |  99.50% | 97.50% | 92.50% | 100.00% |
| detector_ngram2 |  94.82% |  99.50% | 99.00% | 100.00% |  96.00% | 100.00% | 96.00% | 96.00% | 96.00% | 100.00% |  99.00% |  97.00% |  99.50% | 99.00% |  99.00% | 97.50% |  7.00% | 100.00% | 89.50% | 100.00% | 84.00% | 71.50% | 98.00% | 97.00% | 96.50% | 98.50% | 100.00% | 99.00% | 95.50% | 100.00% | 91.50% | 95.50% | 99.00% | 97.50% | 95.00% | 85.00% | 100.00% | 97.50% | 90.50% |  99.50% | 96.50% |  98.50% | 97.50% | 99.50% | 99.50% | 98.00% | 98.50% | 100.00% | 97.00% | 95.00% | 100.00% |
| detector_ngram3 |  95.95% | 100.00% | 99.00% | 100.00% |  98.00% |  99.50% | 98.50% | 97.00% | 96.00% | 100.00% | 100.00% |  99.50% | 100.00% | 99.00% | 100.00% | 98.50% |  8.50% | 100.00% | 89.50% | 100.00% | 90.00% | 85.00% | 98.50% | 98.50% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 92.50% | 96.50% | 99.00% | 99.00% | 96.50% | 91.50% | 100.00% | 99.50% | 90.50% |  99.50% | 97.00% | 100.00% | 98.50% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 97.50% |  99.50% |
| detector_ngram4 |  96.89% | 100.00% | 99.00% | 100.00% |  99.50% | 100.00% | 98.00% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.00% | 30.50% | 100.00% | 89.50% |  99.50% | 93.50% | 92.50% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 94.50% | 98.00% | 99.00% | 99.00% | 97.00% | 95.00% |  99.50% | 99.50% | 91.00% | 100.00% | 97.00% | 100.00% | 98.50% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% |  99.50% |
| detector_ngram5 |  98.01% | 100.00% | 99.50% | 100.00% | 100.00% | 100.00% | 98.00% | 99.00% | 97.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.00% | 78.50% | 100.00% | 90.00% |  99.50% | 94.00% | 93.50% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 96.50% | 100.00% | 97.50% | 99.00% | 99.00% | 99.00% | 97.50% | 94.50% |  99.50% | 99.50% | 91.00% | 100.00% | 97.00% | 100.00% | 99.00% | 99.00% | 99.00% | 98.50% | 98.50% | 100.00% | 96.50% | 99.00% |  99.50% |


In [14]:
showtable(WB, LANGS, threshold=0.5)

50 columns in total, 47 columns left.
|                 | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | heb     | hin    | hun     | ido    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-----------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| detector_ngram1 | 

|                 | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | heb     | hin    | hun     | ido    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-----------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
| detector_ngram1 |  94.00% |  99.50% | 99.00% | 100.00% |  94.00% |  98.50% | 93.50% | 88.50% | 94.50% | 100.00% |  91.00% |  92.00% |  99.50% | 97.50% |  93.00% | 86.00% | 100.00% | 87.00% |  99.00% | 66.50% | 96.50% | 82.00% | 91.00% | 98.00% | 100.00% | 99.00% | 94.00% |  99.00% | 91.00% | 93.00% | 96.00% | 85.00% | 76.50% |  99.00% | 92.00% | 89.00% |  99.50% | 96.50% |  90.00% | 96.50% | 95.00% | 99.50% | 93.00% | 98.50% |  99.50% | 97.50% | 92.50% | 100.00% |
| detector_ngram2 |  97.39% |  99.50% | 99.00% | 100.00% |  96.00% | 100.00% | 96.00% | 96.00% | 96.00% | 100.00% |  99.00% |  97.00% |  99.50% | 99.00% |  99.00% | 97.50% | 100.00% | 89.50% | 100.00% | 84.00% | 98.00% | 97.00% | 96.50% | 98.50% | 100.00% | 99.00% | 95.50% | 100.00% | 91.50% | 95.50% | 99.00% | 97.50% | 95.00% | 100.00% | 97.50% | 90.50% |  99.50% | 96.50% |  98.50% | 97.50% | 99.50% | 99.50% | 98.00% | 98.50% | 100.00% | 97.00% | 95.00% | 100.00% |
| detector_ngram3 |  98.14% | 100.00% | 99.00% | 100.00% |  98.00% |  99.50% | 98.50% | 97.00% | 96.00% | 100.00% | 100.00% |  99.50% | 100.00% | 99.00% | 100.00% | 98.50% | 100.00% | 89.50% | 100.00% | 90.00% | 98.50% | 98.50% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 92.50% | 96.50% | 99.00% | 99.00% | 96.50% | 100.00% | 99.50% | 90.50% |  99.50% | 97.00% | 100.00% | 98.50% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 97.50% |  99.50% |
| detector_ngram4 |  98.44% | 100.00% | 99.00% | 100.00% |  99.50% | 100.00% | 98.00% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.00% | 100.00% | 89.50% |  99.50% | 93.50% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 94.50% | 98.00% | 99.00% | 99.00% | 97.00% |  99.50% | 99.50% | 91.00% | 100.00% | 97.00% | 100.00% | 98.50% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% |  99.50% |
| detector_ngram5 |  98.60% | 100.00% | 99.50% | 100.00% | 100.00% | 100.00% | 98.00% | 99.00% | 97.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.00% | 100.00% | 90.00% |  99.50% | 94.00% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 96.50% | 100.00% | 97.50% | 99.00% | 99.00% | 99.00% | 97.50% |  99.50% | 99.50% | 91.00% | 100.00% | 97.00% | 100.00% | 99.00% | 99.00% | 99.00% | 98.50% | 98.50% | 100.00% | 96.50% | 99.00% |  99.50% |
